## Automated Monitoring - Deployment Monitoring Integration¶

Verta provides a extensible [model monitoring framework](https://docs.verta.ai/verta/monitoring) that allows the user to fully define and configure what data to monitor and how to monitor it.

This is an example to showcase how to use automated monitoring when you deploy models in Verta. Learn more about [automated monitoring for Verta endpoints](https://docs.verta.ai/verta/monitoring/guides/automated-monitored-entities).

The Verta client offers the ability to profile your training data and upload their distributions using log_training_data_profile()to monitor live inputs and outputs against expected thresholds and track missing data.

This notebook provides an example of how to setup and deploy a XGBoost model to use automated monitoring in Verta.

## 0. Imports¶

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import itertools
import time

import six

import numpy as np
import pandas as pd

import sklearn
from sklearn import datasets
from sklearn import model_selection

import xgboost as xgb

## 0.1 Verta import and setup¶

In [ ]:
# restart your notebook if prompted on Colab
try:
    import verta
except ImportError:
    !pip install verta

In [ ]:
# import os
# os.environ['VERTA_EMAIL'] = 
# os.environ['VERTA_DEV_KEY'] = 
# os.environ['VERTA_HOST']

from verta import Client
client = Client(os.environ['VERTA_HOST'])

## 1. Model Training¶

### 1.1 Prepare data¶

In [ ]:
data = datasets.load_wine()

X = data['data']
y = data['target']

dtrain = xgb.DMatrix(X, label=y)

In [ ]:
df = pd.DataFrame(np.hstack((X, y.reshape(-1, 1))),
                  columns=data['feature_names'] + ['species'])

df.head()

In [ ]:
X_train = df.iloc[:,:-1]
y_train = df.iloc[:, -1]

## 1.2 Fit Model¶

In [ ]:
hyperparams =  {
    'eta': 0.5,
    'max_depth' : 2,
    'num_class' : 10
}

model = xgb.XGBClassifier(**hyperparams)
model.fit(X, y)

## 2. Register model for deployment¶


In [ ]:
registered_model = client.get_or_create_registered_model(
    name="monitoring-wine-example-aug17", labels=["xgboost"])

## 2.1 Create model version¶

In [ ]:
from verta.environment import Python
from verta.utils import ModelAPI

model_version = registered_model.create_standard_model_from_xgboost(
    model,
    environment=Python(requirements=["xgboost", "sklearn"]),
    model_api=ModelAPI(X_train, y_train),
    name="v1"
)

## 2.2 Profile training data

Here, Verta uploads profiles of your training data to faciliate downstream monitoring. Individual data points are not uploaded—the client only passes along numerical and categorical distributions of the columns in your data.

This step may take a few moments to run, depending on the dimensionality of your training data.

In [ ]:
model_version.log_training_data_profile(X_train, y_train)

These training data distributions can be viewed in the web app.

In [ ]:
model_version

## 3. Deploy model¶

In [ ]:
wine_endpoint = client.get_or_create_endpoint("monitoring-wine-example-aug17")
wine_endpoint.update(model_version, wait=True)

### 3.1 Make predictions¶

In [ ]:
deployed_model = wine_endpoint.get_deployed_model()
deployed_model.predict([X[0]])

### 3.2 Make few more predictions to create alerts¶

In [ ]:
deployed_model = wine_endpoint.get_deployed_model()
print(deployed_model)
for i, row in X_train.iterrows():
    print ([row.tolist()])
    print(deployed_model.predict([row.tolist()]))
    
    if i == 5:
        break